In [13]:
import requests
from bs4 import BeautifulSoup
import json
import re
import sys
import time, random
import pprint
import urllib3
import csv, codecs

In [14]:
#리스트 및 변수 초기화
nlist=[] 
List=[]
mention = 0

# 쿼리에 검색어 입력, 검색 시작날짜와 끝 날짜 지정
query = "문재인"   # url 인코딩 에러는 encoding parse.quote(query) 
s_date = "2018.08.11" 
e_date = "2018.09.14" 
s_from = s_date.replace(".","") 
e_to = e_date.replace(".","") 

# 저장 엑셀 파일 지정
f = open("C:/Users/user/Desktop/" + query + '뉴스언급량0001.csv', 'w', encoding='cp949', newline='')
wr = csv.writer(f)

In [15]:
def get_news(n_url): 
    global mention
    news_detail = []
    nlist.append(n_url) 
    breq = requests.get(n_url) 
    bsoup = BeautifulSoup(breq.content, 'html.parser') 

    # html 파싱 
    title = bsoup.select('h3#articleTitle')[0].text 
    news_detail.append(title) 

    # 날짜 파싱
    pdate = bsoup.select('.t11')[0].get_text()[:11] 
    news_detail.append(pdate)  
    
    # 기사 본문 크롤링 
    _text = bsoup.select('#articleBodyContents')[0].get_text().replace('\n', " ") 
    btext = _text.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "") 
    news_detail.append(btext.strip())
    
    if(query.count("+") == 0):
        for i in news_detail:
            mention += i.count(query)
    else:
        m=query.split("+")
        for k in m:
            for i in news_detail:
                mention += i.count(k)
    
    return news_detail

In [ ]:
import pandas as pd
import time
from fake_useragent import UserAgent
from selenium.webdriver.chrome import options
from selenium import webdriver

options = options.Options()
options.add_argument(r"--headless")
n_date = s_date
n_date_in = n_date.replace(".","") 
num_pages = 1
while int(n_date_in) < int(e_to):         
    print(n_date)
    num_pages = 1
    while num_pages < 1000:
        prepdate = n_date.replace(".","-")
        # 페이지 지정
        #print(num_pages)
        ua = str(UserAgent().random)
        options.add_argument(r"--{0}".format(ua))
        driver = webdriver.Chrome(r"C:\Users\user\Desktop\chromedriver.exe",
                            chrome_options=options)
        options.arguments.remove(r"--{0}".format(ua))
        url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sort=2&ds=" + n_date + "&de=" + n_date + "&nso=so%3Ar%2Cp%3Afrom" + n_date_in + "to" + n_date_in + "%2Ca%3A&start=" + str(num_pages) 
        #header 추가 
        header = { 
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36', 
        } 
        req = requests.get(url,headers=header) 
        cont = req.content 
        soup = BeautifulSoup(cont, 'html.parser')

        for urls in soup.select("._sp_each_url"): 
            try : 
                if urls["href"].startswith("https://news.naver.com"):
                    get_news(urls["href"]) 
                    news_detail = get_news(urls["href"])
            except Exception as e: 
                print(e)
                continue
            except SSLError as e:
                continue
        num_pages = num_pages + 10
    wr.writerow([n_date, mention])
    if(int(n_date_in) == 20180831): n_date_in = "20180901"
    else: n_date_in=str(int(n_date_in)+1)
    n_date = n_date_in[:4] + '.' + n_date_in[4:6] + '.' + n_date_in[6:]
f.close()
print("Done!")

2018.08.11
list index out of range
list index out of range
